In [2027]:
from cached_property import cached_property
from itertools import starmap, zip_longest
import operator
import numpy as np
from itertools import starmap
from functools import reduce

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

In [2028]:
class Partition:
    def __init__(self, *args):
        args = args or (0,)
        if isinstance(args[0], (list, tuple)):
            self.parts = tuple(sorted(args[0], reverse = True))  # cast as tuple
            
        elif isinstance(args[0], (int, float)):
            self.parts = tuple(args)
            
        self.iter_index = -1
        self.end = len(self.parts)
        
    def __len__(self):
        return len(self.parts)
        
    def __eq__(self, other):
        if isinstance(other, Partition):
            return self.parts == other.parts
        
        else:
            return self.parts == other
    
    def __lt__(self, other):
        return self.parts < other.parts
    
    def __le__(self, other):
        return self.parts <= other.parts
    
    def __gt__(self, other):
        return self.parts > other.parts
    
    def __ge__(self, other):
        return self.parts >= other.parts
    
    def __repr__(self):
        return str(self.parts)
        
    def __str__(self):
        return "\n".join("☐"*part for part in self.parts)
    
    def __add__(self, other):
        other_type = type(other)
        
        if isinstance(other, (Partition)):
            both = zip_longest(self.parts, other.parts, fillvalue = 0)
            
        elif isinstance(other, (list, tuple)):
            both = zip_longest(self.parts, other, fillvalue = 0)
            
        elif isinstance(other, int):
            return Partition((part + other for part in self.parts))
        
        return Partition(starmap(operator.add, both))
    
    def __getitem__(self, ind):
        return self.parts[ind]
    
    def __iter__(self):
        for part in self.parts:
            yield part
        
    def __next__(self):
        if self.iter_index > self.end: 
            raise StopIteration 
            
        else: 
            self.iter_index += 1
            return self.parts[self.iter_index]
        
    def __hash__(self):
        return hash(self.parts)
        
        
    def index(self, val, last = False):
        for ind, part in enumerate(self[::-1]) if last else enumerate(self):
                if part == val:
                    return len(self) - ind - 1 if last else ind
       
        return -1
        
    @staticmethod
    def reverse_insort(a, x, lo=0, hi=None):
        """ Reverse insort version of the bisect.insort method. """
        if lo < 0:
            raise ValueError('lo must be non-negative')
        if hi is None:
            hi = len(a)
        while lo < hi:
            mid = (lo+hi)//2
            if x > a[mid]: hi = mid
            else: lo = mid+1
        a.insert(lo, x)
    
    @cached_property
    def is_stable(self):
        return not np.where(np.diff(self) > -2)[0].size
    
    @cached_property
    def ar_parts(self):
        ar_parts = {}
        ar_starts = set()
        for ind, part in enumerate(self):
            if part in ar_starts:
                continue
                
            end_index = max(Partition(self[ind + 1:]).index(part, last = True), 
                            Partition(self[ind + 1:]).index(part - 1, last = True))

            if end_index >= 0:
                ar_parts[ind] = self[ind: ind + end_index + 2]
                ar_starts.add(part)
                
        return ar_parts
        
    
    @cached_property
    def box_size(self):
        if not self.is_stable:
            return 0
        
        if len(self) == 1:
            return self[0]
        
        return reduce(operator.mul, 
                      starmap(lambda x, y: x - y - 1, 
                               list(zip(self.parts, self.parts[1:]))
                             )
                     ) * self.parts[-1]
            
    @cached_property
    def conjugate(self):
        """ https://en.wikipedia.org/wiki/Partition_(number_theory)#Conjugate_and_self-conjugate_partitions """
        new_conj = []
        parts_copy = list(self.parts)
        
        while parts_copy:
            new_conj.append(len(parts_copy))
            parts_copy = [part - 1 for part in parts_copy if part != 1]
        
        return Partition(new_conj)
        
    
    @cached_property
    def matrix(self):
        """ Simply returns a numpy representation of the matrix in a N x N grid where n is the 
        sum of parts in the given partition instance. """
        grid = np.zeros((self.sum_of_parts, self.sum_of_parts))
        
        for ind, part in enumerate(self.parts):
            grid[ind][:part] = 1
            
        return grid
    
    @cached_property
    def sum_of_parts(self):
        """ Returns the sum of parts of the partition instance. """
        return sum(self.parts)
    
    @cached_property
    def durfee(self):
        """ https://en.wikipedia.org/wiki/Durfee_square """
        if not self.parts:
            return 0
        
        i = 0
        for ind, part in enumerate(self.parts):
            if part < ind + 1:
                break
            i += 1
        
        return i
    
    @cached_property
    def rank(self):
        """ https://en.wikipedia.org/wiki/Rank_of_a_partition """
        if not self.parts:
            return 0
        
        return self.parts[0] - len(self.parts)
    
    @cached_property
    def crank(self):
        """ https://en.wikipedia.org/wiki/Crank_of_a_partition """
        if not self.parts:
            return 0
        
        l = self.parts[0]
        w = self.parts.count(1)
        
        if not w:
            return l
        
        u = len([part for part in self.parts if part > w])
        
        return u - w
        
    @cached_property
    def rp(self):
        """ https://arxiv.org/pdf/1409.2192.pdf (Denoted as 'r sub p' in the paper). """
        return (-np.diff(self.parts).clip(-2, 0).sum() // 2) + 1
    
    def is_almost_rectangular(self):
        """ https://arxiv.org/pdf/1409.2192.pdf """
        if len(self.parts) < 2:
            return False
        
        return (parts[0] - parts[-1]) <= 1
    
    
    @cached_property
    def oblak(self):
        """ https://arxiv.org/pdf/1409.2192.pdf
        Returns the resulting partition after performing the partition Recursive Process as specified in the paper. """
        
        oblak = []
        partition = Partition(self[::])
        
        ar_parts = partition.ar_parts
        
        while ar_parts:
            max_u_chain_start_index = max(ar_parts, key = lambda x: 2*x + sum(ar_parts[x]))
            max_u_chain_value = 2*max_u_chain_start_index + sum(ar_parts[max_u_chain_start_index])
            
            if partition[0] >= max_u_chain_value:
                oblak.append(partition[0])
                partition = Partition(partition[1:])
            
            else:
                partition = Partition(tuple([part - 2 for part in partition[:max_u_chain_start_index]]) + \
                        partition[max_u_chain_start_index + len(ar_parts[max_u_chain_start_index]):])
            
                oblak.append(max_u_chain_value)
                
            ar_parts = partition.ar_parts
            
        for part in partition:
            self.reverse_insort(oblak, part)
            
        return Partition(oblak)
    
    @cached_property
    def _next_oblak_step(self):
        """ This is a property that should only really be used for testing purposes. The only reason this
        exists is for neural network training purposes. """
        
        oblak = []
        partition = Partition(self[::])
        
        ar_parts = partition.ar_parts
        
        if ar_parts:
            max_u_chain_start_index = max(ar_parts, key = lambda x: 2*x + sum(ar_parts[x]))
            max_u_chain_value = 2*max_u_chain_start_index + sum(ar_parts[max_u_chain_start_index])

            if partition[0] >= max_u_chain_value:
                oblak.append(partition[0])
                partition = Partition(partition[1:])

            else:
                partition = Partition(tuple([part - 2 for part in partition[:max_u_chain_start_index]]) + \
                        partition[max_u_chain_start_index + len(ar_parts[max_u_chain_start_index]):])

                oblak.append(max_u_chain_value)

            ar_parts = partition.ar_parts
            
        for part in partition:
            self.reverse_insort(oblak, part)
            
        return Partition(oblak)
        

In [2029]:
class PartitionClass:
    def __init__(self, n = 0):
        self.n = n
         
    def _generate_partitions(self):
        """ This function is about as fast as it gets for generating integer partitions
        without taking advantage of multithreading. """
        a = [0 for i in range(self.n + 1)]
        k = 1
        a[1] = self.n
        while k != 0:
            x = a[k - 1] + 1
            y = a[k] - 1
            k -= 1
            while x <= y:
                a[k] = x
                y -= x
                k += 1
            a[k] = x + y
            yield Partition(a[:k + 1])
       
    @cached_property
    def partitions(self):
        return [p for p in self._generate_partitions()]
            
    def filter_by_attribute(self, attr, val = True):
        assert hasattr(Partition(), attr), f"The Partition class does not have attribute: {attr}."
        
        return [p for p in self._generate_partitions() if getattr(p, attr) == val]
    
    def filter_by_attribute_lists(self, attr, vals):
        assert hasattr(Partition(), attr), f"The Partition class does not have attribute: {attr}."
        
        return [p for p in self._generate_partitions() if getattr(p, attr) in vals]

In [2030]:
class OblakClass(PartitionClass):
    def __init__(self, oblak = Partition()):
        super().__init__(n = oblak.sum_of_parts)
        self.oblak = oblak

    @cached_property
    def partitions(self):
        return [p for p in self._generate_partitions() if p.oblak == self.oblak]

# Training Data

In [1859]:
n = 30

In [1994]:
training_partitions = PartitionClass(n).partitions
training_matrices = [p.matrix for p in training_partitions]
training_durfees = [p.oblak for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_durfees)

partition_encoder = dict(zip(set(y), range(len(set(y)))))
y = np.array([partition_encoder[partition] for partition in y])
partition_decoder = {v: k for k, v in partition_encoder.items()}

label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
# y_train = y_train.reshape(*y_train.shape, -1)
# y_test = y_test.reshape(*y_test.shape, -1)

## Keras Model

In [1997]:
from keras.layers import Input, MaxPooling2D, UpSampling2D
from keras.models import Model

In [1925]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=2, activation="relu", input_shape=(n, n, 1)))
model.add(Conv2D(32, kernel_size=2, activation="relu"))
model.add(Conv2D(16, kernel_size=2, activation="relu"))
model.add(Conv2D(8, kernel_size=2,  activation="relu"))
model.add(Flatten())
model.add(Dense(units=len(y[0]), activation='softmax'))

In [1998]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=2, activation="relu", input_shape=(n, n, 1)))
model.add(Conv2D(32, kernel_size=2, activation="relu"))
model.add(Flatten())
model.add(Dense(units=len(y[0]), activation='softmax'))

In [1999]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [2000]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.0001, patience=300, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [2001]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, callbacks=cbs, batch_size=100, workers = 4)

Train on 4203 samples, validate on 1401 samples
Epoch 1/1000
4203/4203 [==============================] - 7s 2ms/step - loss: 4.4621 - accuracy: 0.0293 - val_loss: 4.1829 - val_accuracy: 0.0557
Epoch 2/1000
2400/4203 [================>.............] - ETA: 2s - loss: 3.8860 - accuracy: 0.0929

KeyboardInterrupt: 


## Next_Oblak_CNN

In [2041]:
n = 28

In [2062]:
training_partitions = PartitionClass(n).partitions
training_matrices = [p.matrix for p in training_partitions]
training_durfees = [p._next_oblak_step.matrix for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_durfees)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
y_train = y_train.reshape(*y_train.shape, -1)
y_test = y_test.reshape(*y_test.shape, -1)

In [2083]:
input_img = Input(shape=(n, n, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_662 (Conv2D)          (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_210 (MaxPoolin (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_663 (Conv2D)          (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_211 (MaxPoolin (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_664 (Conv2D)          (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_212 (MaxPoolin (None, 4, 4, 8)           0  

In [2091]:
input_img = Input(shape=(n, n, 1))

x = Conv2D(16, (4, 4), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (4, 4), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (4, 4), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_718 (Conv2D)          (None, 28, 28, 16)        272       
_________________________________________________________________
max_pooling2d_234 (MaxPoolin (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_719 (Conv2D)          (None, 14, 14, 8)         2056      
_________________________________________________________________
max_pooling2d_235 (MaxPoolin (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_720 (Conv2D)          (None, 7, 7, 8)           1032      
_________________________________________________________________
max_pooling2d_236 (MaxPoolin (None, 4, 4, 8)           0  

In [2092]:
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

In [2093]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.001, patience=300, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [2095]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=cbs, batch_size=100, workers = 4)

Train on 2788 samples, validate on 930 samples
Epoch 1/100
2788/2788 [==============================] - 4s 2ms/step - loss: 0.2576 - accuracy: 0.9626 - val_loss: 0.0534 - val_accuracy: 0.9681
Epoch 2/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0496 - accuracy: 0.9787 - val_loss: 0.0402 - val_accuracy: 0.9811
Epoch 3/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0358 - accuracy: 0.9853 - val_loss: 0.0334 - val_accuracy: 0.9864
Epoch 4/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0324 - accuracy: 0.9863 - val_loss: 0.0254 - val_accuracy: 0.9894
Epoch 5/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0271 - accuracy: 0.9885 - val_loss: 0.0266 - val_accuracy: 0.9891
Epoch 6/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0258 - accuracy: 0.9889 - val_loss: 0.0251 - val_accuracy: 0.9892
Epoch 7/100
2788/2788 [==============================] - 4s 1ms/step - loss: 0.0253

Epoch 57/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0205 - accuracy: 0.9910 - val_loss: 0.0207 - val_accuracy: 0.9910
Epoch 58/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0205 - accuracy: 0.9910 - val_loss: 0.0207 - val_accuracy: 0.9910
Epoch 59/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0204 - accuracy: 0.9911 - val_loss: 0.0206 - val_accuracy: 0.9911
Epoch 60/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0203 - accuracy: 0.9911 - val_loss: 0.0206 - val_accuracy: 0.9911
Epoch 61/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0203 - accuracy: 0.9912 - val_loss: 0.0205 - val_accuracy: 0.9911
Epoch 62/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0202 - accuracy: 0.9912 - val_loss: 0.0204 - val_accuracy: 0.9911
Epoch 63/100
2788/2788 [==============================] - 3s 1ms/step - loss: 0.0201 - accuracy: 0.9912 - val_loss: 0.0203 -

In [2096]:
for partition in PartitionClass(n).partitions:
    model_input = np.array(PartitionClass(n).partitions[0].matrix).reshape(-1, *(n, n, 1))
    prediction = Partition([x.sum() for x in model.predict(model_input)[0].round() if x.sum()])
    
    assert partition.oblak == prediction, (partition, partition.oblak, prediction)

AssertionError: ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (28,), (22.0, 3.0, 1.0))

In [2099]:
validation = [
    {
        'original': partition,
        'oblak': partition.oblak,
        'prediction': Partition([x.sum() for x in model.predict(np.array(partition.matrix).reshape(-1, *(n, n, 1)))[0].round() if x.sum()])
    }
 for partition in PartitionClass(n).partitions
    ]

In [2105]:
sorted(validation, key = lambda x: x['prediction'])[21]

{'original': (10, 9, 9), 'oblak': (28,), 'prediction': (15.0, 5.0, 2.0, 1.0)}

## Validation

In [2049]:
model.evaluate(X_test, y_test, verbose=0)

[0.013035430572926998, 0.9945550560951233]

In [2059]:
model_input = np.array(PartitionClass(n).partitions[0].matrix).reshape(-1, *(n, n, 1))
Partition([x.sum() for x in model.predict(model_input)[0].round() if x.sum()])


(23.0, 1.0)

In [2050]:
for partition in PartitionClass(n).partitions:
    model_input = np.array(partition.matrix).reshape(-1, *(n, n, 1))
    prediction = label_binarizer.inverse_transform(model.predict(model_input))[0]
    
    assert partition.oblak == prediction, (partition.oblak, prediction)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Feature Importance

In [2002]:
n = 30

In [2003]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
def base_model():
    #create model
    model = Sequential()
    #add model layers
    model.add(Conv2D(64, kernel_size=2, activation="relu", input_shape=(n, n, 1)))
    model.add(Conv2D(32, kernel_size=2, activation="relu"))
    model.add(Flatten())
    model.add(Dense(units=len(y[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def build_data(n):
    training_partitions = PartitionClass(n).partitions
    training_matrices = [p.matrix for p in training_partitions]
    training_durfees = [p.oblak for p in training_partitions]

    X = np.array(training_matrices)
    y = np.array(training_durfees)

    partition_encoder = dict(zip(set(y), range(len(set(y)))))
    y = np.array([partition_encoder[partition] for partition in y])
    partition_decoder = {v: k for k, v in partition_encoder.items()}

    label_binarizer = LabelBinarizer()
    y = label_binarizer.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    X_train = X_train.reshape(*X_train.shape, -1)
    X_test = X_test.reshape(*X_test.shape, -1)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = build_data(n)

model = KerasRegressor(build_fn=base_model)

lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.0001, patience=300, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=cbs, batch_size=100, workers = 4)

In [2019]:
perm = PermutationImportance(model, random_state = 1).fit(X_train,y_train)
eli5.show_weights(perm, feature_names = X._traincolumns.tolist())

ValueError: Found array with dim 4. Estimator expected <= 2.

In [ ]:
len(X_train[0].flatten())

In [2010]:
model.loss


AttributeError: 'KerasRegressor' object has no attribute 'loss'

In [1971]:
model.fit(X_train, b
         )

In [1957]:
input_dim = model.input.shape.as_list()